# Animate map data
This notebook demonstrates how to create an animation from a time-series of geophysical data.

In [1]:
output_root = '/home/bhillma/data/acme_scratch/sandiatoss3'
case_name = 'master-tutorial.FC5AV1C-L.ne4_ne4.baseline'
output_file = '%s/%s/run/%s.cam.h0.0001-01-01-00000.nc'%(output_root, case_name, case_name)

Define a function to plot things on a map from unstructured grid output

In [2]:
# For some reason, cartopy does not like longitude coordinates that
# run from 0 to 360, and seems to expect coordinates from -180 to
# 180 instead. I think this is mostly a problem with the tricontour
# and tripcolor routines I use for unstructured data, but I have not
# really explored this too much. I am sure there is an easier way
# around this, but as a cheap fix I just define a function here to
# move our longitude coordinate from (0, 360) to (-180, 180).
def fix_longitude(lon):
    from numpy import where
    new_longitudes = lon.copy(deep=True)
    new_longitudes[:] = where(lon > 180, lon - 360, lon)
    return new_longitudes

def plot_map(longitude, latitude, data, **kwargs):
    from matplotlib import pyplot
    from cartopy import crs
    
    # Use function to fix our longitude coordinate.
    fixed_longitudes = fix_longitude(lon)

    # Get current axes
    ax = pyplot.gca()
    
    # Do the plotting. We can use the "tri" versions of contourf and pcolor
    # to automatically use the matplotlib triangulation library to deal with
    # our unstructured data. Note that cartopy assumes that data passed to
    # contourf and pcolor methods is on the same projection as the axes are.
    # So, if we chose something other than PlateCarree() above, then we need
    # to specify that the data is still in PlateCarree() so that the proper
    # transformation is applied.
    pl = ax.tripcolor(fixed_longitudes, lat, data, transform=crs.PlateCarree())

    # return the plot object
    return pl

Define function to plot one time sample

In [3]:
def animate_map(lon, lat, data, **kwargs):
    # Open a figure and initialize out map axes
    from matplotlib import pyplot
    from cartopy import crs
    figure = pyplot.figure()
    ax = figure.add_subplot(111, projection=crs.PlateCarree())
    ax.coastlines()

    # Define a function here that animates the map by just plotting a single
    # map from a single time sample. This is looped over internally using
    # the matplotlib.animation.FuncAnimation function below
    def animate(i):
        pl = plot_map(lon, lat, data.isel(time=i))

    # Create the animation
    from matplotlib import animation
    ani = animation.FuncAnimation(figure, animate, frames=data.time.size)
    
    return ani

Read some data and plot something

In [4]:
# %matplotlib notebook

# Read data using xarray
from xarray import open_dataset
ds = open_dataset(output_file)
data = ds['FSUTOA'].squeeze()
lat = ds['lat']
lon = ds['lon']

# Create animation
ani = animate_map(lon, lat, data)

# Convert animation to HTML so we can view in the browser here
from IPython.display import HTML
HTML(ani.to_jshtml())

/ascldap/users/bhillma/projects/xarray/xarray/conventions.py:411: RuntimeWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netCDF4.datetime objects instead, reason: dates out of range
  result = decode_cf_datetime(example_value, units, calendar)
/ascldap/users/bhillma/projects/xarray/xarray/conventions.py:430: RuntimeWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netCDF4.datetime objects instead, reason: dates out of range
  calendar=self.calendar)
